In [ ]:
Legge i dati sui progetti proventienti da DPR&DI dal DB Mongo, calcola gli indicatori di interesse e li salva su un file csv.

In [1]:
from pymongo import MongoClient
from pprint import pprint
# import datetime
# from datetime import datetime, timedelta
import pandas as pd
import numpy as np

In [3]:
MONGO_CONN = "***" #stringa di connessione al DB Mongo
client = MongoClient(MONGO_CONN)

In [3]:
dprdiDB = client.DPRDIAHP # Nome del DB

In [4]:
# Collezioni su DB
datiRiassuntivi = dprdiDB.datiRiassuntivi
riepilogoIncentivi = dprdiDB.riepilogoIncentivi

In [5]:
df = pd.DataFrame.from_records(list(datiRiassuntivi.find({},{'_id':0})))

In [7]:
df.dataInizio = df.dataInizio.astype('datetime64[ns]')
df.dataFine = df.dataFine.astype('datetime64[ns]')
df.dataInizioPrimaAttivita = df.dataInizioPrimaAttivita.astype('datetime64[ns]')
df.dataFineUltimaAttivita = df.dataFineUltimaAttivita.astype('datetime64[ns]')

In [8]:
df['durataProgetto'] = df[['dataFineUltimaAttivita', 'dataInizioPrimaAttivita']].apply(lambda x: (x.dataFineUltimaAttivita - x.dataInizioPrimaAttivita).days +1, axis=1)

In [9]:
list_RI_1 = list(riepilogoIncentivi.aggregate(
    [
        {
            '$project' : {
                '_id' : '$idProgetto',
                'totaleFattureIntraProgetto' : '$totaleFattureIntraProgetto',
                'totaleFattureExtraProgetto' : '$totaleFattureExtraProgetto',
                'totaleSpesaPersonaleRS' : '$totaleSpesaPersonaleRS',
                'costoTotaleProgetto' : {'$add' : ['$totaleFattureIntraProgetto', '$totaleFattureExtraProgetto', '$totaleSpesaPersonaleRS']},
            }
        }
    ]
))


In [10]:
list_RI_2 = list(riepilogoIncentivi.aggregate(
[
    {
        '$project': {
            'idProgetto': '$idProgetto', 
            'idSpesa': '$speseRendicontate.idSpesa', 
            'quotaEleggibile': '$speseRendicontate.quotaEleggibile', 
            'consulenza': '$speseRendicontate.rigaFattura.fattura.consulenza'
        }
    }, {
        '$unwind': {
            'path': '$idSpesa', 
            'includeArrayIndex': 'index1'
        }
    }, {
        '$unwind': {
            'path': '$quotaEleggibile', 
            'includeArrayIndex': 'index2'
        }
    }, {
        '$unwind': {
            'path': '$consulenza', 
            'includeArrayIndex': 'index3'
        }
    }, {
        '$project': {
            'idProgetto': 1, 
            'idSpesa': 1, 
            'quotaEleggibile': 1, 
            'consulenza': 1, 
            'valid': {
                '$and': [
                    {
                        '$eq': [
                            '$index1', '$index2'
                        ]
                    }, {
                        '$eq': [
                            '$index1', '$index3'
                        ]
                    }
                ]
            }
        }
    }, {
        '$match': {
            'valid': True, 
            'consulenza': True
        }
    }, {
        '$group': {
            '_id': '$idProgetto', 
            'totaleConsulenze': {
                '$sum': '$quotaEleggibile'
            }
        }
    }
]
))

In [11]:
list_RI_3 = list(riepilogoIncentivi.aggregate(
[
    {
        '$project': {
            '_id': '$idProgetto', 
            'totaleRendicontatoBandi': {
                '$sum': '$_totaliPerVoceBando.totale'
            }
        }
    }
]
))

In [12]:
df_RI_1 = pd.DataFrame(list_RI_1)
df_RI_2 = pd.DataFrame(list_RI_2)
df_RI_3 = pd.DataFrame(list_RI_3)

In [13]:
df = df.merge(df_RI_1, how='outer', left_on = 'idProgetto', right_on = '_id') 

In [15]:
df = df.merge(df_RI_2, how='outer', on = '_id') 

In [16]:
df = df.merge(df_RI_3, how='outer', on = '_id') 

In [17]:
df = df[df.costoTotaleProgetto!=0].copy()

In [18]:
df = df[df.totaleSpesaPersonaleRS!=0].copy()

In [19]:
indicatori_df = pd.DataFrame()
indicatori_df['idProgetto'] = df.idProgetto

In [20]:
# Internal Business Perspective 
indicatori_df['findings_su_costoTotale'] = df.numeroFindings/df.costoTotaleProgetto
indicatori_df['numerosita_su_durata'] = df.numerositaTeam/df.durataProgetto
indicatori_df['rendicontatoBandi'] = df.totaleRendicontatoBandi
indicatori_df['oreTeam_su_numerosita'] = df.oreTotaliTeam/df.numerositaTeam
indicatori_df['oreTeam_su_numeroAttivita'] = df.oreTotaliTeam/df.numeroAttività
# Innovation and Learning Perspective 
indicatori_df['articoli_su_numerosita'] = df.numeroRiviste/df.numerositaTeam
indicatori_df['volumi_su_numerosita'] = df.numeroLibri/df.numerositaTeam
indicatori_df['brevetti_su_costoTotale'] = df.numeroBrevetti/df.costoTotaleProgetto
indicatori_df['findings_su_durata'] = df.numeroFindings/df.durataProgetto
indicatori_df['articoli_su_costoTotale'] = df.numeroRiviste/df.costoTotaleProgetto
indicatori_df['findings_su_oreTeam'] = df.numeroFindings/df.oreTotaliTeam
# Financial Perspective
indicatori_df['costoTeam_su_costoTotale'] = df.totaleSpesaPersonaleRS/df.costoTotaleProgetto
indicatori_df['costoFornitori_su_costoTotale'] = df.totaleFattureExtraProgetto/df.costoTotaleProgetto
# indicatori_df['costoMacchinari_su_costoTotale'] = df./df.costoTotaleProgetto
indicatori_df['rendicBandi_su_costoTotale'] = df.totaleRendicontatoBandi/df.costoTotaleProgetto
indicatori_df['brevetti_su_costoTeam'] = df.numeroBrevetti/df.totaleSpesaPersonaleRS
indicatori_df['oreTeam_su_costoTeam'] = df.oreTotaliTeam/df.totaleSpesaPersonaleRS
# Alliances and networks perspective
indicatori_df['numeroPartner'] = df.numeroPartner
indicatori_df['numeroPartner_su_durata'] = df.numeroPartner/df.durataProgetto
indicatori_df['costoFornitori_su_numeroAttivita'] = df.totaleFattureExtraProgetto/df.numeroAttività
indicatori_df['brevetti_su_consulenti'] = df.numeroBrevetti/df.numeroConsulenti

In [21]:
indicatori_df

,idProgetto,findings_su_costoTotale,numerosita_su_durata,rendicontatoBandi,oreTeam_su_numerosita,oreTeam_su_numeroAttivita,articoli_su_numerosita,volumi_su_numerosita,brevetti_su_costoTotale,findings_su_durata,...,findings_su_oreTeam,costoTeam_su_costoTotale,costoFornitori_su_costoTotale,rendicBandi_su_costoTotale,brevetti_su_costoTeam,oreTeam_su_costoTeam,numeroPartner,numeroPartner_su_durata,costoFornitori_su_numeroAttivita,brevetti_su_consulenti
0,237,0.000029,0.013699,0.000,484.600000,302.875000,0.200000,0.0,0.000000,0.002740,...,0.000413,0.540779,0.000000,0.000000,0.000000,0.131896,2,0.005479,0.000000,0.000000
1,238,0.000000,0.008505,0.000,54.428571,95.250000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.166260,0.548426,0.000000,0.000000,0.063602,1,0.001215,4940.000000,0.000000
2,239,0.000000,0.006840,0.000,308.600000,220.428571,0.000000,0.0,0.000000,0.000000,...,0.000000,0.897785,0.102215,0.000000,0.000000,0.025096,1,0.001368,1000.000000,0.000000
3,240,0.000035,0.028509,24480.960,126.615385,182.888889,0.076923,0.0,0.000000,0.004386,...,0.001215,0.650783,0.349217,0.427458,0.000000,0.044163,1,0.002193,2222.222222,0.000000
5,242,0.000035,0.023585,15000.000,139.600000,174.500000,0.000000,0.0,0.000000,0.009434,...,0.002865,0.328412,0.613036,0.262730,0.000000,0.037227,2,0.009434,8750.000000,0.000000
6,244,0.000000,0.030688,0.000,1220.551724,1769.800000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.674342,0.297745,0.000000,0.000000,0.048839,1,0.001058,16000.000000,0.000000
7,245,0.000005,0.038182,211996.000,271.476190,1140.200000,0.000000,0.0,0.000005,0.001818,...,0.000175,0.452121,0.469010,1.114666,0.000012,0.066300,1,0.001818,17840.000000,0.142857
8,246,0.000000,0.020000,169669.000,491.727273,601.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.655176,0.252871,0.780080,0.000000,0.037957,1,0.001818,6111.111111,0.000000
9,247,0.000033,0.011457,90318.780,470.857143,299.636364,0.142857,0.0,0.000000,0.004910,...,0.000910,0.659379,0.340621,1.000013,0.000000,0.055345,1,0.001637,2796.732727,0.000000
10,249,0.000003,0.109290,536054.000,451.657500,2007.366667,0.025000,0.0,0.000000,0.002732,...,0.000055,1.000000,0.000000,1.493309,0.000000,0.050328,1,0.002732,0.000000,0.000000


In [31]:
indicatori_df.set_index('idProgetto', inplace=True)

In [35]:
indicatori_df.to_csv("indicatori_dprdi.csv")